# Pandas cut material



<!-- PYTHON PREREQUISITE -->
<!-- Pandas is a library built on top of basic Python syntax: -->
<!-- Maybe add a refresher on Python: list, sum, len, indexing, slicing, ??? -->
Pandas is a Python library,
so any prior experience with Python will come in handy.
Remember that Appendix C - [python_tutorial.ipynb](./python_tutorial.ipynb) contains a Python tutorial
you can use to get up to speed quickly on the syntax,
so if you haven't checked that out yet now would be a good time.
<!--
before we really start getting into the Pandas code examples.
This intro-to-python appendix will show you Python data types (`int`, `float`, `bool`, `list`, etc.),
Python operators (`+`, `-`, `*`, `/`, `**`, etc.),
and functions (`len`, `sum`, `print`, etc.).
There is nothing complicated,
but it's good to learn the basic syntax
so you'll know the meaning of symbols like `=`, `:`, `[`, `(`, etc.
-->

In [1]:
import pandas as pd

In [2]:
s = pd.Series([1,2,3,4])

In [3]:
type(s.values)

numpy.ndarray

<!-- NUMPY DIGRESSION -->
Under the hood,
the values of the series use stored in a `numpy` arrays.
<!-- Robyn said: Needs definition? -->


In [4]:
df = pd.read_csv("datasets/minimal.csv")


In [5]:
# OTHER ATTRIBUTES
# df.axes
# df.memory_usage()
# df.values

The column `y` can also be obtained by accessing the attribute `df.y`.

In [6]:
df["y"].equals( df.y )

True

Note accessing columns as attributes only works for column names that do not contain spaces or special characters.
<!-- TODO: say we won't use in this book, but you need to know since other authors may use this syntax -->


In [7]:
# MAYBE
ys = df["y"]
ys.values

array([2. , 1. , 1.5, 2. , 1.5])

In [8]:
# MAYBE
ys[2]

np.float64(1.5)

#### Selecting subsets of the data frame

We can use a combined selection expression to choose
and arbitrary subset of the rows and columns of the data frame.

We rarely need to do this,
but for the purpose of illustration of the `loc` syntax,
here is the code for selecting the `y` and `team` columns
from the last two rows of the data frame.

In [9]:
df.loc[3:5, ["y","team"]]

,y,team
3,2.0,b
4,1.5,b


In [10]:
# add the columns to the index; result is series with a multi-index
# df.stack()

In [11]:
# ALT. way to do transpose
# df.stack().reorder_levels([1,0]).unstack()

### Datasets for the book

In [12]:
students = pd.read_csv("datasets/students.csv")

In [13]:
students.head()

,student_ID,background,curriculum,effort,score
0,1,arts,debate,10.96,75.0
1,2,science,lecture,8.69,75.0
2,3,arts,debate,8.60,67.0
3,4,arts,lecture,7.92,70.3
4,5,science,debate,9.90,76.1


In [14]:
xD = students[students["curriculum"]=="debate"]["score"].values
xL = students[students["curriculum"]=="lecture"]["score"].values

In [15]:
import numpy as np
np.mean(xD), np.mean(xL), np.mean(xD) -np.mean(xL)

(np.float64(76.4625),
 np.float64(68.14285714285714),
 np.float64(8.319642857142867))

In [16]:
from scipy.stats import ttest_ind

ttest_ind(xD, xL)

TtestResult(statistic=np.float64(1.7197867420465698), pvalue=np.float64(0.10917234443214315), df=np.float64(13.0))

## Data pre-processing tasks

- *Extract* the "raw" data from various data source formats
  (spreadsheet, databases, files, web servers).
- *Transform* the data by reshaping and cleaning it.
- *Load* the data into the system used for statistical analysis.

## Extract


### Extract data from different source formats

On UNIX systems (Linux and macOS) the command line program `head` can be used to show the first few lines of any file. The command `head` is very useful for exploring files—by printing the first few lines, you can get an idea of the format it is in.

Unfortunately, on Windows the command `head` is not available, so instead of relying on command line tools, we'll write a simple Python function that called `head` that does the same thing as the command line tool: it prints the first few lines of a file. By default this function will print the first five lines of the file, but users can override the `count` argument to request a different number of lines to be printed.

In [17]:
import os

def head(path, count=7):
    """
    Print the first `count` lines of the file at `path`.
    """
    datafile = open(path, "r")
    lines = datafile.readlines()
    for line in lines[0:count]:
        print(line, end="")


The function `head` contains some special handling for Windows users.
If the path is specified using the UNIX path separator `/`,
it will be auto-corrected to use the Windows path separator `\`.

## Load
ALT. Storing statistical datasets
ALT. Data loading

When working with data,
it's important to follow good practices for storing data and metadata associated with your analysis.
Make sure you always have dataset in a format
that is easy to load into the software system we'll be using for data analysis.

It's still a good idea to save the dataset to a new CSV file in order to separate
the extraction, transformation, and cleaning steps from the subsequent statistical analysis.
Saving the dataset in a general-purpose format like CSV will also make it easy to share the data with collaborators,
or experiment with other statistical software like [RStudio](https://www.rstudio.com/),
[JASP](https://jasp-stats.org/), and [Jamovi](https://www.jamovi.org/).

To save the data frame `cleandf` as the CSV file `mydata.csv`,
we can use its `.to_csv()` method.

Let's save the cleaned data to the file `mydata.csv` in a directory `mydataset`.

In [18]:
# cleandf.to_csv("mydataset/mydata.csv", index=False)

We can verify the data was successfully saved to disk using the `head` function,
TODO remove head( call
which prints the first few lines from the file.

In [19]:
# head("mydataset/mydata.csv")

We should also create a short text file `README.txt` that describes the data file,
and provides the codebook information for the variables.
<!-- Robyn said: Encourage structured/standardized metadata formats -->

<!-- Information about the dataset is provided in a text file `README.txt`. -->

Include in the README any information about the dataset that might be relevant for the research project.

Remember to document all steps we followed to obtain the dataset.
This includes information about how we obtained the data
and the transformation and cleaning steps we performed.
It's important to record all these details in order to make the ETL pipeline reproducible,
meaning someone else could run the same procedure as you to obtain the same dataset.
Ideally,
you should include a Jupyter notebook or Python script with the data transformations.

<!-- Robyn said: Consider explaining the importance of reproducibility - why we don't just manually change values in an excel spreadsheet, for example. -->

In [20]:
# head("mydataset/README.txt", count=11)

## Series

Pandas `Series` objects are list-like containers of values.
We work with series whenever performing calculations on individual columns (variables) of a data frame.
We'll start by creating a standalone `pd.Series` object,
and defer the discussion about data frames until the next subsection.

The code line below shows how to create a series from a list of four numbers.
Pandas `pd.Series` objects are similar to Python lists `[3,5,7,9]`.
They are containers for series of values.

In [21]:
s = pd.Series([3, 5, 7, 9])

We stored the series object into a variable named `s`,
which is short for series.

We can print the series `s` by simply typing its name in a new code cell.
Recall that the notebook interface automatically prints the last expression evaluated in a code cell.

In [22]:
s

0    3
1    5
2    7
3    9
dtype: int64

The numbers printed on the left are called the `index` of the series,
while the numbers on the right are the `values` of the series.
The last line in the output shows some additional information about the series.
The series `s` contains integers,
so its `dtype` (data type) is `int64`.

<!-- Robyn said: This is the first time that dtype and int64 is brought up. Should we explain this more? -->


We use integer indices to identify the elements in the series:
the first element is at index `0`,
the second element is at index `1`,
and so on,
until the last element which is at index `len(s)-1`.
Here is an example that shows accessing individual values of the series
using the default 0-based indexing.

In [23]:
print("First:  index =", 0, " value =", s[0])
print("Second: index =", 1, " value =", s[1])
print("Last:   index =", len(s)-1, " value =", s[len(s)-1])

First:  index = 0  value = 3
Second: index = 1  value = 5
Last:   index = 3  value = 9


The series `index` attribute tells you all the possible indices for the series.

In [24]:
s.index

RangeIndex(start=0, stop=4, step=1)

In [25]:
list(s.index)

[0, 1, 2, 3]

The series `s` uses the default index `[0, 1, 2, 3]`, 
which consists of a range of integers, starting at 0,
just like the index of a Python list with four elements.

The `values` attributes of the series tells you the underlying values without the index.

In [26]:
s.values

array([3, 5, 7, 9])

You can access the individual elements of the series using the square brackets syntax
based on the index labels for the series.
The first element in the series is at index `0`,
so we access it as follows:

In [27]:
s[0]

np.int64(3)

We can select a range of elements from the list using the the square
brackets and slice notation for the indices:

In [28]:
s[0:3]

0    3
1    5
2    7
dtype: int64

The slice notation `0:3` refers to the list of indices `[0,1,2]`.
The result of `s[0:3]` is a new series that contains a subset of the original series
that contains the first three elements.


#### Calculations

Pandas series have methods for performing common calculations.
For example,
the method `.count()` tells us length of the series:


In [29]:
s.count()  # == len(s)

np.int64(4)

The method `.sum()` computes the sum of the values in the series.

<!-- TODO: mention this is the same as SUM(\...) in Excel -->

In [30]:
s.sum()

np.int64(24)

You can perform arithmetic operations like `+`, `-`, `*`, `/` with series.
For example,
we can convert the counts in the series `s` to proportions,
but dividing the series `s` by the sum of the values.


In [31]:
s / s.sum()

0    0.125000
1    0.208333
2    0.291667
3    0.375000
dtype: float64

Series have methods for computing *descriptive statistics* like
`.min()`, `.max()`, `.mean()`, `.median()`, `.var()`, `.std()`, `.quantile()`, etc.
For example,
the mean (average value) and the standard deviation (dispersion from the mean)
are two common statistics we want to compute from data.
We can calculate the arithmetic mean of the values in the series `s` by calling its `.mean()` method.


In [32]:
s.mean()

np.float64(6.0)

To find the sample standard deviation of the values in the series `s`,
we use the `.std()` method.


In [33]:
s.std()

np.float64(2.581988897471611)

Pandas makes it really easy to compute all descriptive statistics!

TODO: TABLE showing all methods on Series objects


We can also use arbitrary `numpy` functions on series,
and Pandas will apply the function to the values in the series.

In [34]:
import numpy as np
np.log(s)

0    1.098612
1    1.609438
2    1.945910
3    2.197225
dtype: float64

#### Bonus material 1

A series can contain `float` values.

In [35]:
s2 = pd.Series([0.3, 1.5, 2.2])
s2

0    0.3
1    1.5
2    2.2
dtype: float64

Here is another example of a series that contains strings (categorical variables).

In [36]:
s3 = pd.Series(["a", "b", "b", "c"])
s3

0    a
1    b
2    b
3    c
dtype: object

#### Bonus material 2

Pandas series allow arbitrary labels to be used as the index, not just integers.
For example, we can use string labels like (`"x"`, `"y"`, etc.) as the index of a series.

In [37]:
s4 = pd.Series(index=["x", "y", "z", "t"],
               data =[ 3,   5,   7,   9 ])
s4

x    3
y    5
z    7
t    9
dtype: int64

In [38]:
s4.index

Index(['x', 'y', 'z', 't'], dtype='object')

In [39]:
s4.values

array([3, 5, 7, 9])

We can now use the string labels to access the individual elements in the series.

In [40]:
s4["y"]

np.int64(5)

In other words,
Pandas series also act like Python `dict`ionary objects with arbitrary keys.
Indeed any quantity that can be used as a `key` in a dictionary (a Python hashable object),
can also be used as a label in a Pandas series.
The list of `keys` of a Python dictionary is the same as the `index` of a Pandas series.

### Collecting the website visitors dataset

Recall the website visitors dataset.
You can see the first few rows of the dataset by loading the CSV file `datasets/visitors.csv`.

In [41]:
visitors = pd.read_csv("datasets/visitors.csv")
visitors.head()

,IP address,version,bought
0,135.185.92.4,A,0
1,14.75.235.1,A,1
2,50.132.244.139,B,0
3,144.181.130.234,A,0
4,90.92.5.100,B,0


Looking at the dataset helps you understand the general structure,
but you have some questions about how the data was collected,
so you decide to call Vanessa and ask her.

"Hi Vanessa. I'm looking at the data you sent me and I had some questions," you start.

"Yes, ask away," she responds.

"How did you assign the visitors to version A or B of the website?" you ask.

"Every time the website received a new visitor (new IP address),
it randomly sent them to either version A or version B," she says.
"It's basically equivalent to flipping a coin."
She goes on to explain that the two versions of the website can be identified from the server logs,
since the two designs use different background images.
A visitor who sees version A of the website will load the background image `images/bgA.jpg`,
while visitors who see version B will load the image `images/bgB.jpg`.

https://riptutorial.com/pandas/example/15180/read-nginx-access-log--multiple-quotechars-

"And how did you calculate the `bought` column?" you ask.

"When a visitor completes the purchase steps,
they are sent to a special `/thankyou` page,
so I used that to identify visitors who bought something."

"Can you tell me more about the steps you took to extract the data from the server logs?"

"Sure," replies Vanessa and starts on a long explanation, which is summarized below.

Vanessa started by extracting the web server access logs for the date range when the experiment was running.
She then loaded the log files into Pandas data frames and concatenated the data from the different days.
<!-- TODO: mention pd commands used for merge -->
She then did some data cleaning by excluding rows generated from bots based on the user agent value.

She then applied the main logic for determining the `bought` variable
by collecting all the log entries for individual IP addresses.
For each visitor (unique IP address) she looked for a request to `images/bgA.jpg`
or `images/bgB.jpg` followed by the request to the `/thankyou` page,
which indicates they bought something (`bought=1`).
If the visitor never reached the `/thankyou` page,
then we know they didn't make a purchase,
so she recorded `bought=0` for them.

Finally she stored the data as `visitors.csv` and sent it to you.

Remore shell  
`zcat /var/log/nginx/access.log.*.gz > /tmp/access_logs.txt`


Local shell  
`scp minireference.com:/tmp/access_logs.txt data/access_logs.txt`

Links

- https://riptutorial.com/pandas/example/15180/read-nginx-access-log--multiple-quotechars-
- [ParseNginxAccessLogs.ipynb](./explorations/ParseNginxAccessLogs.ipynb)

In [42]:
# access_logs = open("data/access_logs.txt")
# df = pd.read_csv(
#     access_logs,
#     sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])',
#     engine='python',
#     usecols=[0, 3, 4, 5, 6, 7, 8],
#     names=['ip', 'time', 'request', 'status', 'size', 'referer', 'user_agent'],
#     na_values='-',
#     header=None
# )

- A VIEWS = CONTAINS /static/images/homepage/bgA.jpg
- B VIEWS = CONTAINS /static/images/homepage/bgB.jpg
- A CONVERSIONS = CONTAINS /static/images/homepage/bgA.jpg and /thankyou
- B CONVERSIONS = CONTAINS /static/images/homepage/bgB.jpg and /thankyou
- p_A = A CONVERSIONS / A VIEWS
- p_B = B CONVERSIONS / B VIEWS


We can compute the conversion rate for each version of the website
using the `.groupby()` method.

In [43]:
visitors.groupby("version") \
         ["bought"].value_counts(normalize=True)

version  bought
A        0         0.935175
         1         0.064825
B        0         0.962229
         1         0.037771
Name: proportion, dtype: float64

In [44]:
visitors.groupby("version") \
         ["bought"].agg(["sum", "count"]) \
         .eval("sum/count")

version
A    0.064825
B    0.037771
dtype: float64

For version A (the new design), the conversion rate is $p_{\!A} = 0.0648$.
For version B (the old design), the conversion rate is $p_{B} = 0.0377$.

<!-- TODO: add conclusion + take home message -->
<!-- TODO: add conversational Vanessa interested in stats --- explain this is an instance of compare two proportions analysis recipe -->


In other words,
Pandas series also act like Python `dict`ionary objects with arbitrary keys.
Indeed any quantity that can be used as a `key` in a dictionary (a Python hashable object),
can also be used as a label in a Pandas series.
The list of `keys` of a Python dictionary is the same as the `index` of a Pandas series.

### Transpose

The *transpose* transformation flips a data frame through the diagonal,
turning the rows into columns, and columns into rows.

In [45]:
dfT = df.transpose()
dfT

,0,1,2,3,4
x,1.0,1.5,2.0,2.5,3.0
y,2.0,1.0,1.5,2.0,1.5
team,a,a,a,b,b
level,3,2,1,3,3


After the transpose operation,
the index `df.index` becomes the column index `dfT.columns`,
while the columns index `df.columns`
becomes the rows index `dfT.index`.

Other data shape transformations methods include
`.melt()`, `.stack()`, `.unstack()`, `.merge()`, etc.
We'll discuss shape-transformation methods later in the tutorial as needed.

Other data shape transformations methods include
`.melt()`, `.stack()`, `.unstack()`, `.merge()`, etc.
We'll discuss shape-transformation methods later in the tutorial as needed.

#### BONUS EXPLAINER 2: UNDO MELT WITH PIVOT

The method for the opposite transformation (converting long data to wide data) is called `pivot` and works like this:

In [46]:
views_data = {
    "season": ["Season 1", "Season 2"],
    "Episode 1": [1000, 10000],
    "Episode 2": [2000, 20000],
    "Episode 3": [3000, 30000],
}
tvwide = pd.DataFrame(views_data)
tvwide

,season,Episode 1,Episode 2,Episode 3
0,Season 1,1000,2000,3000
1,Season 2,10000,20000,30000


In [47]:
tvlong = tvwide.melt(id_vars=["season"],
                     var_name="episode",
                     value_name="views")
tvlong

,season,episode,views
0,Season 1,Episode 1,1000
1,Season 2,Episode 1,10000
2,Season 1,Episode 2,2000
3,Season 2,Episode 2,20000
4,Season 1,Episode 3,3000
5,Season 2,Episode 3,30000


In [48]:
tvlong.pivot(index="season",
             columns="episode",
             values="views")

episode,Episode 1,Episode 2,Episode 3
season,,,
Season 1,1000,2000,3000
Season 2,10000,20000,30000


In [49]:
# ALT. to get *exactly* the same data frame as `tvwide`
tvlong.pivot(index="season",
             columns="episode",
             values="views") \
       .reset_index() \
       .rename_axis(columns=None)

,season,Episode 1,Episode 2,Episode 3
0,Season 1,1000,2000,3000
1,Season 2,10000,20000,30000


## Bonus topics

### Pandas plot methods

Use the `.plot()` method to obtain basic plots,
see Appendix E for Seaborn tutorial for more advanced plots (specific for statistics).


### NumPy arrays

Under the hood, Pandas `Series` and `DataFrame` objects are based on efficient numerical NumPy arrays.
You generally won't need to interact with NumPy commands when working in Pandas,
but sometimes it can be useful to know the NumPy syntax to perform certain data selection tasks.

Let's look at some simple examples
of the data `[1,3,5,7]` stored as a NumPy array.

In [50]:
import numpy as np
values = np.array([1, 3, 5, 7])
values

array([1, 3, 5, 7])

In [51]:
values - 2

array([-1,  1,  3,  5])

In [52]:
np.exp(values)

array([   2.71828183,   20.08553692,  148.4131591 , 1096.63315843])

### Selecting a subset of the values

Selection of subsets is similar to what we do in Pandas:
we use a mask to select the desired values.

In [53]:
values < 4  # selection mask

array([ True,  True, False, False])

In [54]:
values[values < 4]

array([1, 3])

#### Create a list of evenly spaced numbers

We'll use this often when plotting functions.

In [55]:
np.linspace(0, 1, 20)

array([0.        , 0.05263158, 0.10526316, 0.15789474, 0.21052632,
       0.26315789, 0.31578947, 0.36842105, 0.42105263, 0.47368421,
       0.52631579, 0.57894737, 0.63157895, 0.68421053, 0.73684211,
       0.78947368, 0.84210526, 0.89473684, 0.94736842, 1.        ])

In [56]:
rawdf = pd.read_csv("datasets/raw/minimal.csv")
cleandf = rawdf.dropna()
cleandf

,x,y,team,level
0,1.0,2.0,a,3.0
1,1.5,1.0,a,2.0
2,2.0,1.5,a,1.0
4,2.5,2.0,b,3.0
5,3.0,1.5,b,3.0


In [57]:
# Recode level as integer
cleandf = rawdf.dropna().copy()
cleandf["level"] = cleandf["level"].astype(int)
# doesn't work
# cleandf.loc[:,"level"] = cleandf["level"].astype(int)
cleandf

,x,y,team,level
0,1.0,2.0,a,3
1,1.5,1.0,a,2
2,2.0,1.5,a,1
4,2.5,2.0,b,3
5,3.0,1.5,b,3


In [58]:
# ALT. Convert the level column to `int`
cleandf = cleandf.assign(level = cleandf["level"].astype(int))
cleandf

,x,y,team,level
0,1.0,2.0,a,3
1,1.5,1.0,a,2
2,2.0,1.5,a,1
4,2.5,2.0,b,3
5,3.0,1.5,b,3


 -  Web files
    Web servers also work with paths.
    The job of a web server is to respond to requests for different paths.
    For example,
    when your web browser makes a GET request for the path `/datasets/minimal.csv`
    on the server `noBSstats.com`,
    the web server software running on `noBSstats.com`
    will respond by sending back the contents of the file.
    The act of GETing a file from a remote host and saving it to the local file system is
    usually accomplished using the "Save as" operation,
    a terminal command line `wget https://noBSstats.com/datasets/minimal.csv`,
    of a Python script `import requests; response = requests.get("..."); ...`.
    <!-- TODO: finish this example ot make it a reference to some code block -->


-   **Research datasets.** 
    Ask researchers to provide you with raw data files from any research paper.
    You can expect mixed responses,
    but it doesn't hurt to ask.

-   **SQL files.** 
    Structured Query Language is the standard format used to represent data in databases.
    A *database dump* files contain the complete instructions for recreating a database and all the contents in it.
    TODO: mention not meant to work directly -- but load into DB then query to extract CSV.



**Content warning:** In the next few pages, acronyms like SQL, TSV, and JSON will be thrown at you.
I know you're thinking "wow that escalated quickly,"
but we have to get a bit technical to make the knowledge actually useful.

<!-- If we're going to talk about data, then let's make it real.
Let's go get the data wherever it may be:
in Excel files, in text files, in databases, in scientific experiment logbooks, everywhere! -->

#### XML

The eXtensible Markup Language format is used by many structured data formats and APIs.
Here are the first few lines of the XML data file `datasets/formats/minimal.xml`.

```xml
<?xml version='1.0' encoding='utf-8'?>
<players>
  <player>
    <x>1.0</x>
    <y>2.0</y>
    <team>a</team>
    <level>3</level>
  </player>
```

We use the function `pd.read_xml()` to load XML data.

In [59]:
xmldf = pd.read_xml("datasets/formats/minimal.xml")
xmldf.equals(df)

True

The Python libraries `lxml` or `BeautifulSoup` can be used
for more advanced XML parsing and data processing steps (out of scope).